# Develop a CasADi Model of The Steady-State Solar Plant RTO Problem

Replicate the calculations in the Excel spreadsheet implementation and try to solve with CasADi.

- `Solar Plant Optimization of N-Pumps I-O 2025-08-29.xlsm`

In [ ]:
import casadi as cas
import numpy as np
import matplotlib.pyplot as plt

## Pump and Flow Calcualtions

In [ ]:
# Actual speed range: 1000 - rpm
# Scaled speed range: 0.2 - 1.0
PUMP_SPEED_MIN = 1000
PUMP_SPEED_MAX = 2970

def actual_pump_speed_from_scaled(speed_scaled):
    return PUMP_SPEED_MIN + (speed_scaled - 0.2) * (PUMP_SPEED_MAX - PUMP_SPEED_MIN) / 0.8

# Test calculation
assert actual_pump_speed_from_scaled(0.2) == PUMP_SPEED_MIN
assert actual_pump_speed_from_scaled(1.0) == PUMP_SPEED_MAX

In [ ]:
def calculate_pump_and_drive_efficiency(total_flow_rate, actual_pump_speed):
    # Excel formula BI36: =($AD$23/$C$7)*(48.91052-123.18953*($AD$23/$C$7)^0.392747)
    x = total_flow_rate / actual_pump_speed
    return x * (48.91052 - 123.18953 * x ** 0.392747)

flow_rates = np.linspace(50, 90, 41)

plt.figure(figsize=(5, 3))
for pump_speed in [1200, 1800, 2400, 3000]:
    pump_and_drive_efficiency = calculate_pump_and_drive_efficiency(
        flow_rates, pump_speed
    )
    plt.plot(flow_rates, pump_and_drive_efficiency, label=pump_speed)
plt.xlabel('Flow rate (kg/s?)')
plt.ylabel('Efficiency')
plt.grid()
plt.legend(title="Speed (rpm)")
plt.title("Pump and Drive Efficiency Curves")
plt.tight_layout()
plt.show()

In [ ]:
def calculate_pump_fluid_power(total_flow_rate, loop_dp):
    return (total_flow_rate / 3600) * loop_dp

# Test calculation
pump_dp = 629.0104406
total_flow_rate = 124.8296578
pump_fluid_power = calculate_pump_fluid_power(total_flow_rate, pump_dp)
assert np.isclose(pump_fluid_power, 21.81087724)

In [ ]:
def calculate_collector_flow_rate(valve_position, loop_dp, b=0.04, c=0.4, sqrt=np.sqrt):
    # Excel formula AD8: =D8*SQRT(($AC$23)/($F$4+$F$5*D8^2))
    return valve_position * sqrt(loop_dp / (b + c * valve_position ** 2))

# Test calculation
valve_position = 0.801565285
loop_dp = 36.8775886
collector_flow_rate = calculate_collector_flow_rate(valve_position, loop_dp)
assert np.isclose(collector_flow_rate, 8.931818991)

In [ ]:
valve_positions = np.linspace(0.1, 1.0, 91)

plt.figure(figsize=(5, 3))
for loop_dp in [50, 60, 70]:
    flow_rate = calculate_collector_flow_rate(valve_positions, loop_dp)
    plt.plot(valve_positions, flow_rate, label=loop_dp)
plt.xlabel('Valve position (-)')
plt.ylabel('Flow rate (kg/s)')
plt.grid()
plt.legend(title="Loop dp (kPa)")
plt.title("Collector Flow Rate Curves")
plt.tight_layout()
plt.show()

In [ ]:
def calculate_total_flowrate(
    valve_positions, loop_dp, sum=np.sum, sqrt=np.sqrt
):
    flow_rates = calculate_collector_flow_rate(
        valve_positions, loop_dp, sqrt=sqrt
    )
    return sum(flow_rates)

# Test calculation
valve_positions = np.array([
    1.0, 0.801565285, 0.681025772, 0.597212957, 0.534172722, 
    0.484242137, 0.443217825, 0.408573903, 0.801565286, 0.681025772, 
    0.597212957, 0.534172722, 0.484242137, 0.443217825, 0.408573903
])
loop_dp = 36.8775886
total_flowrate = calculate_total_flowrate(valve_positions, loop_dp)
assert np.isclose(total_flowrate, 124.8296578)

In [ ]:
BOILER_FLOW_LOSS_FACTOR = 0.038

def calculate_boiler_dp(total_flow_rate):
    # Excel formula G19: =$H$4*F23^2
    return BOILER_FLOW_LOSS_FACTOR * total_flow_rate ** 2

# Test calculation
total_flow_rate = 124.8296578
boiler_dp = calculate_boiler_dp(total_flow_rate)
assert np.isclose(boiler_dp, 592.132852)

In [ ]:
PUMP_DP_MAX = 1004.2368
PUMP_QMAX = 224.6293
EXPONENT = 4.346734

def calculate_pump_dp(actual_pump_speed, total_flow_rate, m_pumps):
    # Excel formula G21: =$C$3*(($C$7/$C$5)^2)*(1-(F23*$C$5/($F$1*$C$4*$C$7))^$C$6)
    return (
        PUMP_DP_MAX * ((actual_pump_speed / PUMP_SPEED_MAX) ** 2) 
        * (
            1 - (
                total_flow_rate * PUMP_SPEED_MAX 
                / (m_pumps * PUMP_QMAX * actual_pump_speed)
            ) ** EXPONENT
        )
    )

# Test calculation
actual_pump_speed = 2362.776896
m_pumps = 2
pump_dp = calculate_pump_dp(actual_pump_speed, total_flow_rate, m_pumps)
assert np.isclose(pump_dp, 629.0104406)

In [ ]:
def calculate_pressure_balance(loop_dp, pump_dp, boiler_dp):
    # AE23: =AC23-(AE21-AE19)
    return loop_dp - (pump_dp - boiler_dp)

# Test calculation
loop_dp = 36.8775886
pump_dp = 629.0104406
boiler_dp = 592.132852
pressure_balance = calculate_pressure_balance(loop_dp, pump_dp, boiler_dp)

assert np.isclose(pressure_balance, 0.0)

In [ ]:
def calculate_total_flowrate(
    valve_positions, loop_dp, sum=np.sum, sqrt=np.sqrt
):
    flow_rates = calculate_collector_flow_rate(
        valve_positions, loop_dp, sqrt=sqrt
    )
    return sum(flow_rates)

# Test calculation
valve_positions = np.array([
    1.0, 0.801565285, 0.681025772, 0.597212957, 0.534172722, 
    0.484242137, 0.443217825, 0.408573903, 0.801565286, 0.681025772, 
    0.597212957, 0.534172722, 0.484242137, 0.443217825, 0.408573903
])
loop_dp = 36.8775886
total_flow_rate = calculate_total_flowrate(valve_positions, loop_dp)
assert np.isclose(total_flow_rate, 124.8296578), total_flow_rate

pump_speed_scaled = 0.75341194
actual_pump_speed = actual_pump_speed_from_scaled(pump_speed_scaled)
assert np.isclose(actual_pump_speed, 2362.776896), actual_pump_speed

pump_dp = 629.0104404750698
pump_fluid_power = calculate_pump_fluid_power(total_flow_rate, pump_dp)
assert np.isclose(pump_fluid_power, 21.81087724), pump_fluid_power

m_pumps = 2
pump_and_drive_efficiency = calculate_pump_and_drive_efficiency(
    total_flow_rate, actual_pump_speed
)
assert np.isclose(pump_and_drive_efficiency, 0.533387324), pump_and_drive_efficiency

pump_and_drive_power = pump_fluid_power / pump_and_drive_efficiency
assert np.isclose(pump_and_drive_power, 40.89125531), pump_and_drive_power

In [ ]:
def make_pressure_balance_function(n_lines, m_pumps, sum=cas.sum, sqrt=cas.sqrt):

    valve_positions = cas.SX.sym('v', n_lines)
    pump_speed_scaled = cas.SX.sym('s')
    loop_dp = cas.SX.sym('dp')

    actual_pump_speed = actual_pump_speed_from_scaled(pump_speed_scaled)

    total_flow_rate = calculate_total_flowrate(
        valve_positions, loop_dp, sum=sum, sqrt=sqrt
    )

    boiler_dp = calculate_boiler_dp(total_flow_rate)

    pump_dp = calculate_pump_dp(actual_pump_speed, total_flow_rate, m_pumps)

    pressure_balance = calculate_pressure_balance(loop_dp, pump_dp, boiler_dp)

    return cas.Function(
        "pressure_balance", 
        [valve_positions, pump_speed_scaled, loop_dp], 
        [pressure_balance], 
        ["valve_positions", "pump_speed_scaled", "loop_dp"], 
        ["pressure_balance"]
    )



def make_system_equations():
    

    pump_and_drive_efficiency = \
        calculate_pump_and_drive_efficiency(
            total_flow_rate, actual_pump_speed
        )

    pump_and_drive_power = pump_fluid_power / pump_and_drive_efficiency


n_lines = 15
m_pumps = 2
pressure_balance_function = make_pressure_balance_function(n_lines, m_pumps)
pressure_balance_function

In [ ]:
# Test calculation
valve_positions = np.array([
    1.0, 0.801565285, 0.681025772, 0.597212957, 0.534172722, 
    0.484242137, 0.443217825, 0.408573903, 0.801565286, 0.681025772, 
    0.597212957, 0.534172722, 0.484242137, 0.443217825, 0.408573903
])
pump_speed_scaled = 0.75341194
loop_dp = 36.8775886

pressure_balance = pressure_balance_function(
    valve_positions, pump_speed_scaled, loop_dp
)
assert np.isclose(pressure_balance, [[0.0]], atol=1e-5)

## Use Rootfinder to Find Pressure Balance

In [ ]:
def g(x):
    loop_dp = x
    return pressure_balance_function(
        cas.DM(valve_positions), cas.DM(pump_speed_scaled), loop_dp
    )

x = cas.SX.sym('x')
rf = cas.rootfinder('rf', 'newton', {'x': x, 'g': g(x)})

x_sol_rf = rf([30.0], [])
x_sol_rf

In [ ]:
# Turn the solution of the root-finder solution into a function

def make_calculate_pump_and_drive_power_function(
    n_lines, m_pumps, sum=cas.sum, sqrt=cas.sqrt
):

    valve_positions = cas.SX.sym('v', n_lines)
    pump_speed_scaled = cas.SX.sym('s')

    pressure_balance_function = make_pressure_balance_function(
        n_lines, m_pumps, sum=sum, sqrt=sqrt
    )

    def g(x):
        loop_dp = x
        return pressure_balance_function(
            valve_positions, pump_speed_scaled, loop_dp
        )
    
    # Make rootfinder to solve pressure balance
    x = cas.SX.sym('x')
    p = cas.vertcat(valve_positions, pump_speed_scaled)
    residual = pressure_balance_function(valve_positions, pump_speed_scaled, x)
    rf = cas.rootfinder('RF', 'newton', {'x': x, 'p': p, 'g': residual})

    # Root finder solution
    sol_rf = rf(x0=[30.0], p=p)
    loop_dp = sol_rf['x']

    flow_rates = calculate_collector_flow_rate(
        valve_positions, loop_dp, sqrt=sqrt
    )

    total_flow_rate = sum(flow_rates)

    actual_pump_speed = actual_pump_speed_from_scaled(pump_speed_scaled)

    pump_and_drive_efficiency = calculate_pump_and_drive_efficiency(
        total_flow_rate, actual_pump_speed
    )

    pump_dp = calculate_pump_dp(actual_pump_speed, total_flow_rate, m_pumps)

    pump_fluid_power = calculate_pump_fluid_power(total_flow_rate, pump_dp)

    pump_and_drive_power = pump_fluid_power / pump_and_drive_efficiency

    return cas.Function(
        "calculate_pump_and_drive_power", 
        [valve_positions, pump_speed_scaled], 
        [pump_and_drive_power],
        ["valve_positions", "pump_speed_scaled"], 
        ["pump_and_drive_power"]
    )

pump_and_drive_power_function = \
    make_calculate_pump_and_drive_power_function(n_lines, m_pumps)
pump_and_drive_power_function

In [ ]:
# Test calculation
valve_positions = np.array([
    1.0, 0.801565285, 0.681025772, 0.597212957, 0.534172722, 
    0.484242137, 0.443217825, 0.408573903, 0.801565286, 0.681025772, 
    0.597212957, 0.534172722, 0.484242137, 0.443217825, 0.408573903
])
pump_speed_scaled = 0.75341194
pump_and_drive_power_function(valve_positions, pump_speed_scaled)
assert np.isclose(pump_and_drive_power, 40.89125531), pump_and_drive_power

In [ ]:
%timeit pump_and_drive_power_function(valve_positions, pump_speed_scaled)

## Temperature and Power Generation Calculations

In [ ]:
GENERATOR_EFFICIENCY = 0.85

LOOP_THERMAL_EFFICIENCY = [
    0.9, 0.88, 0.86, 0.84, 0.82, 0.8, 0.78, 0.76,
    0.88, 0.86, 0.84, 0.82, 0.8, 0.78, 0.76
]